### Without C.V
* Still takes 17 hours to run!

In [ ]:
import os
import numpy as np
import pydicom
import cv2
import tensorflow as tf
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns

# Directories for SIIM-ACR Pneumothorax Segmentation dataset
train_image_dir = r'C:\Users\Jaber\OneDrive - University of Florida\Educational\GitHub\Datasets\ImageSegmentation\SIIM-ACR Pneumothorax Segmentation\archive\pneumothorax\dicom-images-train'
test_image_dir = r'C:\Users\Jaber\OneDrive - University of Florida\Educational\GitHub\Datasets\ImageSegmentation\SIIM-ACR Pneumothorax Segmentation\archive\pneumothorax\dicom-images-test'
train_csv_path = r'C:\Users\Jaber\OneDrive - University of Florida\Educational\GitHub\Datasets\ImageSegmentation\SIIM-ACR Pneumothorax Segmentation\archive\pneumothorax\train-rle.csv'

img_size = (256, 256)

# Load the combined CSV that contains both train and test mask information
combined_df = pd.read_csv(train_csv_path)

# Split into training and test sets based on the availability of the image files
train_df = combined_df[combined_df['ImageId'].apply(lambda x: os.path.exists(os.path.join(train_image_dir, x + '.dcm')))]
test_df = combined_df[combined_df['ImageId'].apply(lambda x: os.path.exists(os.path.join(test_image_dir, x + '.dcm')))]

# Define the rle2mask function here instead of importing from mask_functions
def rle2mask(rle, width, height):
    mask = np.zeros(width * height, dtype=np.uint8)
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]
    current_position = 0
    for start, length in zip(starts, lengths):
        current_position += start
        mask[current_position:current_position + length] = 255
        current_position += length
    return mask.reshape((height, width))

# Data generator to load data in batches
def data_generator(image_dir, df, img_size, batch_size=16):
    while True:
        df_shuffled = df.sample(frac=1).reset_index(drop=True)
        for start in range(0, len(df_shuffled), batch_size):
            end = min(start + batch_size, len(df_shuffled))
            batch_df = df_shuffled.iloc[start:end]

            images = []
            masks = []

            for index, row in batch_df.iterrows():
                img_id = row['ImageId']
                img_path = os.path.join(image_dir, img_id + '.dcm')

                dicom_data = pydicom.dcmread(img_path)
                img = dicom_data.pixel_array

                img = cv2.resize(img, img_size)
                img = img / 255.0  # Normalize image to range 0-1

                # Check if there is a mask
                if pd.isna(row['EncodedPixels']):
                    mask = np.zeros(img_size)  # No pneumothorax, empty mask
                else:
                    mask = rle2mask(row['EncodedPixels'], dicom_data.Rows, dicom_data.Columns)
                    mask = cv2.resize(mask, img_size)
                    mask = mask / 255.0  # Normalize mask to range 0-1

                images.append(np.expand_dims(img, axis=-1))  # Add channel dimension to the image
                masks.append(np.expand_dims(mask, axis=-1))  # Add channel dimension to the mask

            yield np.array(images), np.array(masks)

# Capsule Layer with Dynamic Routing
from tensorflow.keras import layers

def squash(vectors, axis=-1):
    """Squashing function to ensure output vectors' lengths are between 0 and 1"""
    s_squared_norm = tf.reduce_sum(tf.square(vectors), axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / tf.sqrt(s_squared_norm + tf.keras.backend.epsilon())
    return scale * vectors

class CapsuleLayer(layers.Layer):
    def __init__(self, num_capsules, dim_capsule, num_routing=3, **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsules = num_capsules
        self.dim_capsule = dim_capsule
        self.num_routing = num_routing

    def build(self, input_shape):
        self.W = self.add_weight(shape=[input_shape[-1], self.num_capsules * self.dim_capsule],
                                 initializer='glorot_uniform', trainable=True)

    def call(self, inputs):
        inputs = tf.reshape(inputs, [-1, inputs.shape[1] * inputs.shape[2], inputs.shape[3]])
        u_hat = tf.einsum('...ij,jk->...ik', inputs, self.W)
        u_hat = tf.reshape(u_hat, [-1, inputs.shape[1], self.num_capsules, self.dim_capsule])
        
        b = tf.zeros(shape=[tf.shape(inputs)[0], inputs.shape[1], self.num_capsules])
        for i in range(self.num_routing):
            c = tf.nn.softmax(b, axis=-1)
            s = tf.reduce_sum(c[..., tf.newaxis] * u_hat, axis=1)
            v = squash(s)
            if i < self.num_routing - 1:
                b += tf.reduce_sum(u_hat * v[:, tf.newaxis, :, :], axis=-1)
        return v

# U-Net with Capsule Network Layers and Dynamic Routing
def unet_capsule_model(input_size=(256, 256, 1)):
    inputs = tf.keras.layers.Input(input_size)
    
    # Contracting Path with Capsules
    c1 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    c1 = CapsuleLayer(num_capsules=8, dim_capsule=16)(c1)
    c1_flattened = tf.keras.layers.Flatten()(c1)  # Flatten the capsule output
    c1_reshaped = tf.keras.layers.Dense(256*256, activation='relu')(c1_flattened)  # Fully connected layer to reshape
    c1_reshaped = tf.keras.layers.Reshape((256, 256, 1))(c1_reshaped)  # Reshape to 4D
    p1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c1_reshaped)
    
    c2 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same')(p1)
    c2 = CapsuleLayer(num_capsules=16, dim_capsule=32)(c2)
    c2_flattened = tf.keras.layers.Flatten()(c2)
    c2_reshaped = tf.keras.layers.Dense(128*128, activation='relu')(c2_flattened)
    c2_reshaped = tf.keras.layers.Reshape((128, 128, 1))(c2_reshaped)
    p2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c2_reshaped)
    
    c3 = tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same')(p2)
    c3 = CapsuleLayer(num_capsules=32, dim_capsule=64)(c3)
    c3_flattened = tf.keras.layers.Flatten()(c3)
    c3_reshaped = tf.keras.layers.Dense(64*64, activation='relu')(c3_flattened)
    c3_reshaped = tf.keras.layers.Reshape((64, 64, 1))(c3_reshaped)
    p3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c3_reshaped)
    
    # Bottleneck
    b = tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same')(p3)
    b = tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same')(b)
    
    # Expansive Path
    u1 = tf.keras.layers.Conv2DTranspose(256, 2, strides=(2, 2), padding='same')(b)
    u1 = tf.keras.layers.concatenate([u1, c3_reshaped])
    c4 = tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same')(u1)
    c4 = tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same')(c4)
    
    u2 = tf.keras.layers.Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(c4)
    u2 = tf.keras.layers.concatenate([u2, c2_reshaped])
    c5 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same')(u2)
    c5 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same')(c5)
    
    u3 = tf.keras.layers.Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(c5)
    u3 = tf.keras.layers.concatenate([u3, c1_reshaped])
    c6 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(u3)
    c6 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(c6)
    
    outputs = tf.keras.layers.Conv2D(1, 1, activation='sigmoid')(c6)
    
    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
    return model

# Dice and Binary Crossentropy combined loss function
def combined_dice_bce_loss(y_true, y_pred):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    dice_loss = 1 - (2. * intersection + 1) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1)
    
    bce_loss = tf.keras.losses.binary_crossentropy(y_true_f, y_pred_f)
    
    return dice_loss + bce_loss

# Learning rate scheduler
def lr_scheduler(epoch, lr):
    if epoch > 10:
        lr = lr * 0.5  # Reduce learning rate after 10 epochs
    return lr

# Early stopping callback to avoid overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

# Batch size for data generator
batch_size = 16

# Train generator and test generator
train_generator = data_generator(train_image_dir, train_df, img_size, batch_size=batch_size)
test_generator = data_generator(test_image_dir, test_df, img_size, batch_size=batch_size)

# Model training and testing
model = unet_capsule_model()
model.compile(optimizer='adam', loss=combined_dice_bce_loss, metrics=['accuracy'])

history = model.fit(train_generator, steps_per_epoch=len(train_df) // batch_size, epochs=50, 
                    validation_data=test_generator, validation_steps=len(test_df) // batch_size, 
                    callbacks=[early_stopping, lr_callback], verbose=1)

# Evaluate on the test set
y_pred = model.predict(test_generator, steps=len(test_df) // batch_size)
threshold = 0.5
y_pred = (y_pred > threshold).astype(np.uint8)
y_test_binary = np.array([mask for _, mask in test_generator]).astype(np.uint8)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test_binary.flatten(), y_pred.flatten())
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.title(f"Confusion Matrix")
plt.show()

# Visualization: Show input image, true mask, and predicted mask for a few samples
for i in range(3):  # Visualize first 3 predictions
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    
    X_test = next(test_generator)[0]
    
    ax[0].imshow(X_test[i].squeeze(), cmap='gray')
    ax[0].set_title('Input Image')
    
    ax[1].imshow(y_test_binary[i].squeeze(), cmap='gray')
    ax[1].set_title('True Mask')
    
    ax[2].imshow(y_pred[i].squeeze(), cmap='gray')
    ax[2].set_title('Predicted Mask')
    
    plt.show()

# Performance report
accuracy = accuracy_score(y_test_binary.flatten(), y_pred.flatten())
recall = recall_score(y_test_binary.flatten(), y_pred.flatten())
precision = precision_score(y_test_binary.flatten(), y_pred.flatten())
f1 = f1_score(y_test_binary.flatten(), y_pred.flatten())
tn, fp, fn, tp = confusion_matrix(y_test_binary.flatten(), y_pred.flatten()).ravel()
specificity = tn / (tn + fp)

print(f'Accuracy: {accuracy:.4f}')
print(f'Recall (Sensitivity): {recall:.4f}')
print(f'Precision: {precision:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Specificity: {specificity:.4f}')



Epoch 1/50
  1/723 ━━━━━━━━━━━━━━━━━━━━ 17:17:56 86s/step - accuracy: 0.2112 - loss: 1.6889

### Train only on One Epoch to reduce runtime

In [ ]:
import os
import numpy as np
import pydicom
import cv2
import tensorflow as tf
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns

# Directories for SIIM-ACR Pneumothorax Segmentation dataset
train_image_dir = r'C:\Users\Jaber\OneDrive - University of Florida\Educational\GitHub\Datasets\ImageSegmentation\SIIM-ACR Pneumothorax Segmentation\archive\pneumothorax\dicom-images-train'
test_image_dir = r'C:\Users\Jaber\OneDrive - University of Florida\Educational\GitHub\Datasets\ImageSegmentation\SIIM-ACR Pneumothorax Segmentation\archive\pneumothorax\dicom-images-test'
train_csv_path = r'C:\Users\Jaber\OneDrive - University of Florida\Educational\GitHub\Datasets\ImageSegmentation\SIIM-ACR Pneumothorax Segmentation\archive\pneumothorax\train-rle.csv'

img_size = (256, 256)

# Load the combined CSV that contains both train and test mask information
combined_df = pd.read_csv(train_csv_path)

# Split into training and test sets based on the availability of the image files
train_df = combined_df[combined_df['ImageId'].apply(lambda x: os.path.exists(os.path.join(train_image_dir, x + '.dcm')))]
test_df = combined_df[combined_df['ImageId'].apply(lambda x: os.path.exists(os.path.join(test_image_dir, x + '.dcm')))]

# Define the rle2mask function here instead of importing from mask_functions
def rle2mask(rle, width, height):
    mask = np.zeros(width * height, dtype=np.uint8)
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]
    current_position = 0
    for start, length in zip(starts, lengths):
        current_position += start
        mask[current_position:current_position + length] = 255
        current_position += length
    return mask.reshape((height, width))

# Data generator to load data in batches
def data_generator(image_dir, df, img_size, batch_size=16):
    while True:
        df_shuffled = df.sample(frac=1).reset_index(drop=True)
        for start in range(0, len(df_shuffled), batch_size):
            end = min(start + batch_size, len(df_shuffled))
            batch_df = df_shuffled.iloc[start:end]

            images = []
            masks = []

            for index, row in batch_df.iterrows():
                img_id = row['ImageId']
                img_path = os.path.join(image_dir, img_id + '.dcm')

                dicom_data = pydicom.dcmread(img_path)
                img = dicom_data.pixel_array

                img = cv2.resize(img, img_size)
                img = img / 255.0  # Normalize image to range 0-1

                # Check if there is a mask
                if pd.isna(row['EncodedPixels']):
                    mask = np.zeros(img_size)  # No pneumothorax, empty mask
                else:
                    mask = rle2mask(row['EncodedPixels'], dicom_data.Rows, dicom_data.Columns)
                    mask = cv2.resize(mask, img_size)
                    mask = mask / 255.0  # Normalize mask to range 0-1

                images.append(np.expand_dims(img, axis=-1))  # Add channel dimension to the image
                masks.append(np.expand_dims(mask, axis=-1))  # Add channel dimension to the mask

            yield np.array(images), np.array(masks)

# Capsule Layer with Dynamic Routing
from tensorflow.keras import layers

def squash(vectors, axis=-1):
    """Squashing function to ensure output vectors' lengths are between 0 and 1"""
    s_squared_norm = tf.reduce_sum(tf.square(vectors), axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / tf.sqrt(s_squared_norm + tf.keras.backend.epsilon())
    return scale * vectors

class CapsuleLayer(layers.Layer):
    def __init__(self, num_capsules, dim_capsule, num_routing=3, **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsules = num_capsules
        self.dim_capsule = dim_capsule
        self.num_routing = num_routing

    def build(self, input_shape):
        self.W = self.add_weight(shape=[input_shape[-1], self.num_capsules * self.dim_capsule],
                                 initializer='glorot_uniform', trainable=True)

    def call(self, inputs):
        inputs = tf.reshape(inputs, [-1, inputs.shape[1] * inputs.shape[2], inputs.shape[3]])
        u_hat = tf.einsum('...ij,jk->...ik', inputs, self.W)
        u_hat = tf.reshape(u_hat, [-1, inputs.shape[1], self.num_capsules, self.dim_capsule])
        
        b = tf.zeros(shape=[tf.shape(inputs)[0], inputs.shape[1], self.num_capsules])
        for i in range(self.num_routing):
            c = tf.nn.softmax(b, axis=-1)
            s = tf.reduce_sum(c[..., tf.newaxis] * u_hat, axis=1)
            v = squash(s)
            if i < self.num_routing - 1:
                b += tf.reduce_sum(u_hat * v[:, tf.newaxis, :, :], axis=-1)
        return v

# U-Net with Capsule Network Layers and Dynamic Routing
def unet_capsule_model(input_size=(256, 256, 1)):
    inputs = tf.keras.layers.Input(input_size)
    
    # Contracting Path with Capsules
    c1 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    c1 = CapsuleLayer(num_capsules=8, dim_capsule=16)(c1)
    c1_flattened = tf.keras.layers.Flatten()(c1)  # Flatten the capsule output
    c1_reshaped = tf.keras.layers.Dense(256*256, activation='relu')(c1_flattened)  # Fully connected layer to reshape
    c1_reshaped = tf.keras.layers.Reshape((256, 256, 1))(c1_reshaped)  # Reshape to 4D
    p1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c1_reshaped)
    
    c2 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same')(p1)
    c2 = CapsuleLayer(num_capsules=16, dim_capsule=32)(c2)
    c2_flattened = tf.keras.layers.Flatten()(c2)
    c2_reshaped = tf.keras.layers.Dense(128*128, activation='relu')(c2_flattened)
    c2_reshaped = tf.keras.layers.Reshape((128, 128, 1))(c2_reshaped)
    p2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c2_reshaped)
    
    c3 = tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same')(p2)
    c3 = CapsuleLayer(num_capsules=32, dim_capsule=64)(c3)
    c3_flattened = tf.keras.layers.Flatten()(c3)
    c3_reshaped = tf.keras.layers.Dense(64*64, activation='relu')(c3_flattened)
    c3_reshaped = tf.keras.layers.Reshape((64, 64, 1))(c3_reshaped)
    p3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c3_reshaped)
    
    # Bottleneck
    b = tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same')(p3)
    b = tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same')(b)
    
    # Expansive Path
    u1 = tf.keras.layers.Conv2DTranspose(256, 2, strides=(2, 2), padding='same')(b)
    u1 = tf.keras.layers.concatenate([u1, c3_reshaped])
    c4 = tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same')(u1)
    c4 = tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same')(c4)
    
    u2 = tf.keras.layers.Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(c4)
    u2 = tf.keras.layers.concatenate([u2, c2_reshaped])
    c5 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same')(u2)
    c5 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same')(c5)
    
    u3 = tf.keras.layers.Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(c5)
    u3 = tf.keras.layers.concatenate([u3, c1_reshaped])
    c6 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(u3)
    c6 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(c6)
    
    outputs = tf.keras.layers.Conv2D(1, 1, activation='sigmoid')(c6)
    
    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
    return model

# Dice and Binary Crossentropy combined loss function
def combined_dice_bce_loss(y_true, y_pred):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    dice_loss = 1 - (2. * intersection + 1) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1)
    
    bce_loss = tf.keras.losses.binary_crossentropy(y_true_f, y_pred_f)
    
    return dice_loss + bce_loss

# Batch size for data generator
batch_size = 16

# Train generator and test generator
train_generator = data_generator(train_image_dir, train_df, img_size, batch_size=batch_size)
test_generator = data_generator(test_image_dir, test_df, img_size, batch_size=batch_size)

# Model training and testing
model = unet_capsule_model()
model.compile(optimizer='adam', loss=combined_dice_bce_loss, metrics=['accuracy'])

history = model.fit(train_generator, steps_per_epoch=len(train_df) // batch_size, epochs=1,  # Train for 1 epoch
                    validation_data=test_generator, validation_steps=len(test_df) // batch_size, 
                    verbose=1)

# Evaluate on the test set
y_pred = model.predict(test_generator, steps=len(test_df) // batch_size)
threshold = 0.5
y_pred = (y_pred > threshold).astype(np.uint8)
y_test_binary = np.array([mask for _, mask in test_generator]).astype(np.uint8)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test_binary.flatten(), y_pred.flatten())
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.title(f"Confusion Matrix")
plt.show()

# Visualization: Show input image, true mask, and predicted mask for a few samples
for i in range(3):  # Visualize first 3 predictions
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    
    X_test = next(test_generator)[0]
    
    ax[0].imshow(X_test[i].squeeze(), cmap='gray')
    ax[0].set_title('Input Image')
    
    ax[1].imshow(y_test_binary[i].squeeze(), cmap='gray')
    ax[1].set_title('True Mask')
    
    ax[2].imshow(y_pred[i].squeeze(), cmap='gray')
    ax[2].set_title('Predicted Mask')
    
    plt.show()

# Performance report
accuracy = accuracy_score(y_test_binary.flatten(), y_pred.flatten())
recall = recall_score(y_test_binary.flatten(), y_pred.flatten())
precision = precision_score(y_test_binary.flatten(), y_pred.flatten())
f1 = f1_score(y_test_binary.flatten(), y_pred.flatten())
tn, fp, fn, tp = confusion_matrix(y_test_binary.flatten(), y_pred.flatten()).ravel()
specificity = tn / (tn + fp)

print(f'Accuracy: {accuracy:.4f}')
print(f'Recall (Sensitivity): {recall:.4f}')
print(f'Precision: {precision:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Specificity: {specificity:.4f}')



642/723 ━━━━━━━━━━━━━━━━━━━━ 1:14:39 55s/step - accuracy: 0.9950 - loss: 1.0624